Original code for fake news

In [30]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 1

upperframe=[]  
for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))
    filename="NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date, Source, Label\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date,Source,Label))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date','Source','Label'])
data.head()

processing page : 1
https://www.politifact.com/factchecks/list/?page=1
30


,Statement,Link,Date,Source,Label
0,"“Putin bans 5G across Russia, destroys all tow...",https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Bloggers,false
1,"Video showing celebrities, politicians and oth...",https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Facebook posts,pants-fire
2,Country singer Jason Aldean sued “The View” co...,https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Facebook posts,false
3,"""WEF chairman Klaus Schwab openly calls for AI...",https://www.politifact.com/factchecks/2023/aug...,"ugust 9, 2023",Instagram posts,false
4,Douglass Mackey was convicted “because he made...,https://www.politifact.com/factchecks/2023/aug...,"ugust 9, 2023",Charlie Kirk,barely-true


Original code for real news

In [31]:
import json

dictionary = {
  "cnn": {
    "link": "http://edition.cnn.com/"
  },
  "bbc": {
    "rss": "http://feeds.bbci.co.uk/news/rss.xml",
    "link": "http://www.bbc.com/"
  },
  "theguardian": {
    "rss": "https://www.theguardian.com/uk/rss",
    "link": "https://www.theguardian.com/international"
  },
  "breitbart": {
    "link": "http://www.breitbart.com/"
  },
  "infowars": {
    "link": "https://www.infowars.com/"
  },
  "foxnews": {
    "link": "http://www.foxnews.com/"
  },
  "nbcnews": {
    "link": "http://www.nbcnews.com/"
  },
  "washingtonpost": {
    "rss": "http://feeds.washingtonpost.com/rss/world",
    "link": "https://www.washingtonpost.com/"
  },
  "theonion": {
    "link": "http://www.theonion.com/"
  }
}

json_object = json.dumps(dictionary, indent = 4) 
with open("NewsPapers.json", "w") as outfile: 
    outfile.write(json_object) 
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

# Set the limit for number of articles to download
LIMIT = 14500

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

count = 1

# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                newsPaper['articles'].append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 100:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            newsPaper['articles'].append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

# Finally it saves the articles as a JSON-file.
try:
    with open('scraped_articles.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)
with open('scraped_articles.json') as json_data:
    d = json.load(json_data)
for i, site in enumerate((list(d['newspapers']))):
    print(i, site)


import pandas as pd
for i, site in enumerate((list(d['newspapers']))):
    articles = list(d['newspapers'][site]['articles'])
    if i == 0:
        df = pd.DataFrame.from_dict(articles)
        df["site"] = site
    else:
        new_df = pd.DataFrame.from_dict(articles)
        new_df["site"] = site
        df = pd.concat([df, new_df], ignore_index = True)     
df.shape
(1844, 5)
df

Building site for  cnn
1  Article has date of type None...
2  Article has date of type None...
3 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2023/08/09/us/how-to-help-hawaii-wildfire-victims-iyw/index.html
4 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2023/08/10/asia/china-jailed-australian-anchor-intl/index.html
5 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/sport/live-news/netherlands-spain-japan-sweden-womens-world-cup-quarterfinal/index.html
6 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/2023/08/11/americas/ecuador-villavicencio-assassination-suspects-colombian-intl-hnk/index.html
7 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/travel/viral-icon-of-the-seas-cruise-ship-photo/index.html
8 articles downloaded from cnn  using newspaper, url:  http://edition.cnn.com/style/article/malaysia-swatch-ban-rainbow-lgbt-watch-intl-hnk/index

KeyboardInterrupt: 

Scraping code modifucation trial

In [27]:
import feedparser as fp
import newspaper
from newspaper import Article
from datetime import datetime
import pandas as pd

# Dictionary of news sites
sites = {
    "cnn": {
        "link": "http://edition.cnn.com/"
    },
    "bbc": {
        "rss": "http://feeds.bbci.co.uk/news/rss.xml",
        "link": "http://www.bbc.com/"
    },
    "theguardian": {
        "rss": "https://www.theguardian.com/uk/rss",
        "link": "https://www.theguardian.com/international"
    },
    "breitbart": {
        "link": "http://www.breitbart.com/"
    },
    "infowars": {
        "link": "https://www.infowars.com/"
    },
    "foxnews": {
        "link": "http://www.foxnews.com/"
    },
    "nbcnews": {
        "link": "http://www.nbcnews.com/"
    },
    "washingtonpost": {
        "rss": "http://feeds.washingtonpost.com/rss/world",
        "link": "https://www.washingtonpost.com/"
    },
    "theonion": {
        "link": "http://www.theonion.com/"
    }
}

LIMIT = 100
data = []

# Iterate through each news company
for site, info in sites.items():
    print("Scraping articles from", site)

    if 'rss' in info:
        d = fp.parse(info['rss'])
        for entry in d.entries:
            if len(data) >= LIMIT:
                break
            if hasattr(entry, 'published'):
                article = {
                    'site': site,
                    'link': entry.link,
                    'published': datetime.fromtimestamp(mktime(entry.published_parsed)).isoformat()
                }
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                    article['title'] = content.title
                    article['text'] = content.text
                    data.append(article)
                except Exception as e:
                    print(e, "continuing...")
                    continue
    else:
        paper = newspaper.build(info['link'], memoize_articles=False)
        for content in paper.articles:
            if len(data) >= LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e, "continuing...")
                continue
            if content.publish_date is None:
                continue
            article = {
                'site': site,
                'title': content.title,
                'text': content.text,
                'link': content.url,
                'published': content.publish_date.isoformat()
            }
            data.append(article)

# Create DataFrame
df = pd.DataFrame(data)

# Print DataFrame shape
print(df.shape)
# Save DataFrame as CSV
df.to_csv('news_articles.csv', index=False)

Scraping articles from cnn
Article `download()` failed with 404 Client Error: Not Found for url: https://money.cnn.com/2023/08/11/economy/uk-economy-gdp-q2/index.html on URL https://money.cnn.com/2023/08/11/economy/uk-economy-gdp-q2/index.html continuing...
Article `download()` failed with 404 Client Error: Not Found for url: https://money.cnn.com/2023/08/11/business/subway-india-cheese-slices-inflation/index.html on URL https://money.cnn.com/2023/08/11/business/subway-india-cheese-slices-inflation/index.html continuing...
Article `download()` failed with 404 Client Error: Not Found for url: https://money.cnn.com/2023/08/11/investing/china-country-garden-loss-warning-intl-hnk/index.html on URL https://money.cnn.com/2023/08/11/investing/china-country-garden-loss-warning-intl-hnk/index.html continuing...
Article `download()` failed with 404 Client Error: Not Found for url: https://money.cnn.com/2023/08/10/media/writers-strike-negotiations-meeting/index.html on URL https://money.cnn.com/2

In [14]:
import sys
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

articlesToScrape = 500

upperframe = []

for page in range(1, (articlesToScrape // 50) + 1):
    print('processing page:', page)
    url = 'https://www.politifact.com/factchecks/list/?page=' + str(page)

    try:
        page = requests.get(url)
    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()
        print('ERROR FOR LINK:', url)
        print(error_type, 'Line:', error_info.tb_lineno)
        continue
    time.sleep(2)
    
    soup = BeautifulSoup(page.text, 'html.parser')
    frame = []
    links = soup.find_all('li', attrs={'class': 'o-listicle__item'})
    
    for j in links:
        Statement = j.find("div", attrs={'class': 'm-statement__quote'}).text.strip()
        Link = "https://www.politifact.com" + j.find("div", attrs={'class': 'm-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div', attrs={'class': 'm-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class': 'm-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs={'class': 'm-statement__content'}).find('img', attrs={'class': 'c-image__original'}).get('alt').strip()
        frame.append((Statement, Link, Date, Source, Label))

    upperframe.extend(frame)

    if len(upperframe) >= articlesToScrape:
        break

data = pd.DataFrame(upperframe, columns=['Statement', 'Link', 'Date', 'Source', 'Label'])
data.head()

processing page: 1
processing page: 2
processing page: 3
processing page: 4
processing page: 5
processing page: 6
processing page: 7
processing page: 8
processing page: 9
processing page: 10


,Statement,Link,Date,Source,Label
0,"“Putin bans 5G across Russia, destroys all tow...",https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Bloggers,false
1,"Video showing celebrities, politicians and oth...",https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Facebook posts,pants-fire
2,Country singer Jason Aldean sued “The View” co...,https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Facebook posts,false
3,"""WEF chairman Klaus Schwab openly calls for AI...",https://www.politifact.com/factchecks/2023/aug...,"ugust 9, 2023",Instagram posts,false
4,Douglass Mackey was convicted “because he made...,https://www.politifact.com/factchecks/2023/aug...,"ugust 9, 2023",Charlie Kirk,barely-true


In [15]:
data

,Statement,Link,Date,Source,Label
0,"“Putin bans 5G across Russia, destroys all tow...",https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Bloggers,false
1,"Video showing celebrities, politicians and oth...",https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Facebook posts,pants-fire
2,Country singer Jason Aldean sued “The View” co...,https://www.politifact.com/factchecks/2023/aug...,"gust 10, 2023",Facebook posts,false
3,"""WEF chairman Klaus Schwab openly calls for AI...",https://www.politifact.com/factchecks/2023/aug...,"ugust 9, 2023",Instagram posts,false
4,Douglass Mackey was convicted “because he made...,https://www.politifact.com/factchecks/2023/aug...,"ugust 9, 2023",Charlie Kirk,barely-true
...,...,...,...,...,...
295,Jamie Foxx tras una coagulación de sangre en e...,https://www.politifact.com/factchecks/2023/jun...,"June 7, 2023",Facebook posts,false
296,“New tapes” of Nancy Pelosi show that the Jan....,https://www.politifact.com/factchecks/2023/jun...,"June 6, 2023",Benny Johnson,false
297,Photos show Target selling children’s clothing...,https://www.politifact.com/factchecks/2023/jun...,"June 6, 2023",Facebook posts,false
298,"Disney ""acaba de publicar"" que no puedes entra...",https://www.politifact.com/factchecks/2023/jun...,"June 6, 2023",Facebook posts,false


In [19]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

# Set the number of articles to scrape
articles_to_scrape = 200

# Create an empty list to store the scraped data
data = []

# Loop through pages until desired number of articles is reached
for page in range(1, (articles_to_scrape // 50) + 1):
    print('Processing page:', page)
    
    # Construct the URL for the page
    url = f'https://www.politifact.com/factchecks/list/?page={page}'

    try:
        # Get the page content
        page = requests.get(url)
    except Exception as e:
        print('Error fetching URL:', url)
        continue
    
    # Pause for a moment to avoid overwhelming the server
    time.sleep(2)
    
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Find all the list items containing article information
    articles = soup.find_all('li', class_='o-listicle__item')
    
    # Loop through each article
    for article in articles:
        # Extract information from the article
        statement = article.find('div', class_='m-statement__quote').text.strip()
        link = "https://www.politifact.com" + article.find('a')['href'].strip()
        date = article.find('footer').text[-14:-1].strip()
        source = article.find('div', class_='m-statement__meta').find('a').text.strip()
        label = j.find('div', attrs={'class': 'm-statement__content'}).find('img', attrs={'class': 'c-image__original'}).get('alt').strip()
        
        # Append the article information to the data list
        data.append((statement, link, date, source, label))

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Statement', 'Link', 'Date', 'Source', 'Label'])

# Display the first few rows of the DataFrame
print(df.head())


Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
                                           Statement  \
0  “Putin bans 5G across Russia, destroys all tow...   
1  Video showing celebrities, politicians and oth...   
2  Country singer Jason Aldean sued “The View” co...   
3  "WEF chairman Klaus Schwab openly calls for AI...   
4  Douglass Mackey was convicted “because he made...   

                                                Link           Date  \
0  https://www.politifact.com/personalities/blog-...  gust 10, 2023   
1  https://www.politifact.com/personalities/faceb...  gust 10, 2023   
2  https://www.politifact.com/personalities/faceb...  gust 10, 2023   
3  https://www.politifact.com/personalities/insta...  ugust 9, 2023   
4  https://www.politifact.com/personalities/charl...  ugust 9, 2023   

            Source  Label  
0         Bloggers  false  
1   Facebook posts  false  
2   Facebook posts  false  
3  Instagram posts  false  
4     Charlie Kirk  

In [20]:
df

,Statement,Link,Date,Source,Label
0,"“Putin bans 5G across Russia, destroys all tow...",https://www.politifact.com/personalities/blog-...,"gust 10, 2023",Bloggers,false
1,"Video showing celebrities, politicians and oth...",https://www.politifact.com/personalities/faceb...,"gust 10, 2023",Facebook posts,false
2,Country singer Jason Aldean sued “The View” co...,https://www.politifact.com/personalities/faceb...,"gust 10, 2023",Facebook posts,false
3,"""WEF chairman Klaus Schwab openly calls for AI...",https://www.politifact.com/personalities/insta...,"ugust 9, 2023",Instagram posts,false
4,Douglass Mackey was convicted “because he made...,https://www.politifact.com/personalities/charl...,"ugust 9, 2023",Charlie Kirk,false
...,...,...,...,...,...
115,“Miss USA boycotts Miss Universe pageant: ‘I’m...,https://www.politifact.com/personalities/faceb...,"July 17, 2023",Facebook posts,false
116,AMC is intentionally deterring people from see...,https://www.politifact.com/personalities/tikto...,"July 14, 2023",TikTok posts,false
117,“Joe Biden announced that the U.S. dollar will...,https://www.politifact.com/personalities/insta...,"July 14, 2023",Instagram posts,false
118,“No human is allowed to travel to Antarctica.”,https://www.politifact.com/personalities/insta...,"July 14, 2023",Instagram posts,false


In [26]:
articles[1]

<article class="sc-1pw4fyi-6 hGCNFC sc-1oawalh-4 dPimIa js_post_item" data-id="1850725546"><div class="sc-1pw4fyi-4 ljPZBC"><a aria-label="Link to DeSantis Has Surprisingly Smooth Verbal Exchange With Iowa State Fair Corn Dog" class="sc-1out364-0 dPMosf sc-1pw4fyi-5 hQxRDX js_link" data-ga='[["Home page click","Modular Curation Story Click - slot 1 - story 2 of 8","https://www.theonion.com/desantis-has-surprisingly-smooth-verbal-exchange-with-i-1850725546"]]' href="https://www.theonion.com/desantis-has-surprisingly-smooth-verbal-exchange-with-i-1850725546" target="_self" title="DeSantis Has Surprisingly Smooth Verbal Exchange With Iowa State Fair Corn Dog"><div class="sc-1dm5z0l-1 hPYqxf"><div class="sc-1dm5z0l-0 ewYewO"><picture class="sc-epkw7d-0 bHizmP"><source media="(max-width: 37.31em)" srcset="https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,g_center,h_362,q_60,w_645/702e9c19f7b1c45f4807a4c53c11db93.jpg" type="image/jpeg"/><source media="(min-width: 37.37em) and (

In [28]:
import newspaper
from newspaper import Article
import pandas as pd

# Set the number of articles to scrape
articles_to_scrape = 200

# Create an empty list to store the scraped data
data = []

# Loop through pages until the desired number of articles is reached
for page in range(1, (articles_to_scrape // 50) + 1):
    print('Processing page:', page)
    
    # Construct the URL for the page
    url = f'https://www.politifact.com/factchecks/list/?page={page}'
    
    # Use the newspaper library to extract article information
    paper = newspaper.build(url, memoize_articles=False)
    
    for article in paper.articles:
        if len(data) >= articles_to_scrape:
            break
        
        try:
            # Download and parse the article
            article.download()
            article.parse()
        except Exception as e:
            print('Error parsing article:', e)
            continue
        
        # Extract information from the article
        statement = article.title
        link = article.source_url
        date = article.publish_date.strftime('%Y-%m-%d') if article.publish_date else ''
        source = 'Politifact'
        label = 'Label'  # Add a method to extract the label if available
        
        # Append the article information to the data list
        data.append((statement, link, date, source, label))

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Statement', 'Link', 'Date', 'Source', 'Label'])

# Display the first few rows of the DataFrame
print(df.head())


Processing page: 1
Error parsing article: Article `download()` failed with 404 Client Error: Not Found for url: https://www.politifact.com/article/2018/feb/12/principles-truth-o-meter-politifacts-methodology-i/%20 on URL https://www.politifact.com/article/2018/feb/12/principles-truth-o-meter-politifacts-methodology-i/ 
Processing page: 2
Error parsing article: Article `download()` failed with 404 Client Error: Not Found for url: https://www.politifact.com/article/2018/feb/12/principles-truth-o-meter-politifacts-methodology-i/%20 on URL https://www.politifact.com/article/2018/feb/12/principles-truth-o-meter-politifacts-methodology-i/ 
Processing page: 3
Processing page: 4
                                           Statement  \
0  Political Extremism in the Public Square: A re...   
1  Fabricated story about Vladimir Putin banning ...   
2  Are black eyes on newsmakers evidence they’re ...   
3  Story about Jason Aldean suing Whoopi Goldberg...   
4  World Economic Forum’s Klaus Schwab d

In [29]:
combined_data = df_real.merge(df_fake)

NameError: name 'df_real' is not defined